# Live Trading Demo

Demonstrates configuring a NautilusTrader TradingNode for live trading
with either MetaTrader 5 or Interactive Brokers.

In [ ]:
import sys, os
sys.path.append(os.path.abspath("../.."))

from trader import (
    GotobiStrategy,
    GotobiConfig,
    RsiMacdMaStrategy,
    RsiMacdMaConfig,
)


## Option A: MetaTrader 5 Live Node

Requires MetaTrader 5 terminal running on Windows.

This example can run either `GotobiStrategy` or `RsiMacdMaStrategy`.


In [ ]:
from nautilus_trader.config import TradingNodeConfig, LiveDataEngineConfig, LiveExecEngineConfig
from nautilus_trader.live.node import TradingNode

from trader.adapters.metatrader import (
    MetaTrader5DataClientConfig,
    MetaTrader5ExecClientConfig,
    MetaTrader5LiveDataClientFactory,
    MetaTrader5LiveExecClientFactory,
)

# Configure MT5 connection
mt5_data_config = MetaTrader5DataClientConfig(
    mt5_login=12345678,
    mt5_password="your_password",
    mt5_server="YourBroker-Demo",
    bar_seconds=60,
    poll_interval=1.0,
)

mt5_exec_config = MetaTrader5ExecClientConfig(
    mt5_login=12345678,
    mt5_password="your_password",
    mt5_server="YourBroker-Demo",
    mt5_deviation=20,
)

# Choose strategy
USE_RSI_MACD_MA = True

if USE_RSI_MACD_MA:
    strategy_config = RsiMacdMaConfig(
        instrument_id="USDJPY.MT5",
        bar_type="USDJPY.MT5-1-MINUTE-MID-EXTERNAL",
        trade_size=1.0,
        contract_size=100_000,
        rsi_period=14,
        rsi_oversold=30.0,
        rsi_overbought=70.0,
        macd_fast=12,
        macd_slow=26,
        macd_signal=9,
        ma_fast=20,
        ma_slow=50,
    )
    strategy = RsiMacdMaStrategy(config=strategy_config)
else:
    strategy_config = GotobiConfig(
        instrument_id="USDJPY.MT5",
        bar_type="USDJPY.MT5-1-MINUTE-MID-EXTERNAL",
        entry_time="01:30:00",
        exit_time="08:30:00",
        trade_size=1.0,
        contract_size=100_000,
    )
    strategy = GotobiStrategy(config=strategy_config)

# Build and run node
node_config = TradingNodeConfig(
    data_clients={"MT5": mt5_data_config},
    exec_clients={"MT5": mt5_exec_config},
)

node = TradingNode(config=node_config)
node.add_data_client_factory("MT5", MetaTrader5LiveDataClientFactory)
node.add_exec_client_factory("MT5", MetaTrader5LiveExecClientFactory)

node.trader.add_strategy(strategy)

# Uncomment to run:
# node.build()
# node.run()


## Option B: Interactive Brokers Live Node

Requires TWS or IB Gateway running.

This example can run either `GotobiStrategy` or `RsiMacdMaStrategy`.


In [ ]:
from nautilus_trader.adapters.interactive_brokers.factories import (
    InteractiveBrokersLiveDataClientFactory,
    InteractiveBrokersLiveExecClientFactory,
)
from trader.adapters.ibkr import ibkr_data_config, ibkr_exec_config

# Configure IBKR connection (paper trading)
ib_data = ibkr_data_config(host="127.0.0.1", port=7497, client_id=1)
ib_exec = ibkr_exec_config(host="127.0.0.1", port=7497, client_id=1, account="DU1234567")

# Choose strategy
IB_USE_RSI_MACD_MA = True

if IB_USE_RSI_MACD_MA:
    ib_strategy_config = RsiMacdMaConfig(
        instrument_id="USD/JPY.IDEALPRO",
        bar_type="USD/JPY.IDEALPRO-1-MINUTE-MID-EXTERNAL",
        trade_size=1.0,
        contract_size=100_000,
        rsi_period=14,
        rsi_oversold=30.0,
        rsi_overbought=70.0,
        macd_fast=12,
        macd_slow=26,
        macd_signal=9,
        ma_fast=20,
        ma_slow=50,
    )
    ib_strategy = RsiMacdMaStrategy(config=ib_strategy_config)
else:
    ib_strategy_config = GotobiConfig(
        instrument_id="USD/JPY.IDEALPRO",
        bar_type="USD/JPY.IDEALPRO-1-MINUTE-MID-EXTERNAL",
        entry_time="01:30:00",
        exit_time="08:30:00",
        trade_size=1.0,
        contract_size=100_000,
    )
    ib_strategy = GotobiStrategy(config=ib_strategy_config)

# Build node
ib_node_config = TradingNodeConfig(
    data_clients={"IB": ib_data},
    exec_clients={"IB": ib_exec},
)

ib_node = TradingNode(config=ib_node_config)
ib_node.add_data_client_factory("IB", InteractiveBrokersLiveDataClientFactory)
ib_node.add_exec_client_factory("IB", InteractiveBrokersLiveExecClientFactory)

ib_node.trader.add_strategy(ib_strategy)

# Uncomment to run:
# ib_node.build()
# ib_node.run()
